<a href="https://colab.research.google.com/github/audreyemmely/pln/blob/main/pln_lista2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Resolução Lista 2**

---

Audrey Emmely Rodrigues Vasconcelos

Karen Nayara Gomes da Silva

In [1]:
import pandas as pd
import re
import nltk
import numpy

In [2]:
nltk.download('stopwords')
nltk.download('rslp')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
df = pd.read_csv('train_data.txt', sep =':::', header = None, engine = 'python', nrows = 10000)
df.columns = ['id', 'title', 'genre', 'description']

###### Questão 1
Determine a distribuição de comprimentos dos textos (em quantidade de caracteres), listando estas quantidades e plotando um histograma. 

###### Questão 2
Aplique os seguintes passos de pré-processamento aos textos: 
* Remova todas as palavras que contêm números; 
* Converta as palavras para minúsculas; 
* Remova pontuação; 
* Tokenize os textos em palavras, gerando um dicionário único com *n* tokens e convertendo cada texto em um vetor de dimensão *n* com a respectiva contagem de palavras. 

Em seguida, encontre as palavras mais frequentes da base de textos. 


In [4]:
def preprocess(description):
  description = re.sub(r'\w*\d\w*', '', description) #remove todas as palavras que contêm números
  description = re.sub(r'[^a-zA-ZáéíóúÁÉÍÓÚâêîôÂÊÎÔãõÃÕçÇ ]', '', description.lower()) #converte as palavras para minúsculas e remove a pontuação
  return re.sub(r'\s+', ' ', description) #retira espaços repetidos

In [5]:
df.description = df.description.apply(preprocess)

In [6]:
df.description

0        listening in to a conversation between his do...
1        a brother and sister with a past incestuous r...
2        as the bus empties the students for their fie...
3        to help their unemployed father make ends mee...
4        the films title refers not only to the unreco...
                              ...                        
9995     the show began with jared and kristen in kris...
9996     the center characters of the film are two kid...
9997     ashland is the story of ash slavin a lonely a...
9998     railroad foreman murray sinclair is dismissed...
9999     cruise the radio airwaves to assist the ghr g...
Name: description, Length: 10000, dtype: object

In [7]:
df['tokens'] = df.description.apply(lambda t: nltk.word_tokenize(t, language='portuguese'))
tokenSet = sorted(set([token for tokenList in df.tokens.values for token in tokenList]))
tokenToIndex = {token: i for i, token in enumerate(tokenSet)}
corpusMatrix = numpy.zeros(shape=(df.shape[0], len(tokenToIndex)))

for i, row in df.iterrows():
  for token in row.tokens:
    corpusMatrix[i][tokenToIndex[token]] += 1

mostFrequent = numpy.argmax(corpusMatrix, axis=1)

In [10]:
df['most_frequents'] = [tokenSet[i] for i in mostFrequent]
df[['description', 'most_frequents']] 

,description,most_frequents
0,listening in to a conversation between his do...,to
1,a brother and sister with a past incestuous r...,a
2,as the bus empties the students for their fie...,the
3,to help their unemployed father make ends mee...,the
4,the films title refers not only to the unreco...,the
...,...,...
9995,the show began with jared and kristen in kris...,the
9996,the center characters of the film are two kid...,the
9997,ashland is the story of ash slavin a lonely a...,his
9998,railroad foreman murray sinclair is dismissed...,and


###### Questão 3
Aplique os seguintes passos de pré-processamento aos textos processados na questão anterior:

* Remova stopwords;
* Realize rotulação de POS;
* Realize stemização;

Exiba os resultados em alguns textos. Verifique quais são as palavras mais
frequentes e compare com as palavras mais frequentes da questão anterior.

In [8]:
stopwords = nltk.corpus.stopwords.words('portuguese')
stemmer = nltk.stem.RSLPStemmer()

**Referências**

[NLTK HOWTO](https://www.nltk.org/howto/portuguese_en.html)

[Examples for Portuguese Processing](https://docs.huihoo.com/nltk/0.9.5/guides/portuguese_en.html)